<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/KNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.under_sampling import RandomUnderSampler
import joblib

In [4]:
treated_data_RN = pd.read_csv('/home/erlon/AdministracaoPublica/CadUnico/BasesRN/cadunico_rn_cleaned.csv', sep=';', encoding='utf-8')

In [5]:
treated_data_RN.dropna(inplace=True)
treated_data_RN.drop_duplicates(inplace=True)

In [6]:
treated_data_RN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296062 entries, 0 to 296061
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  296062 non-null  int64  
 1   classf                      296062 non-null  int64  
 2   vlr_renda_media_fam         296062 non-null  float64
 3   cod_local_domic_fam         296062 non-null  float64
 4   qtd_comodos_domic_fam       296062 non-null  float64
 5   qtd_comodos_dormitorio_fam  296062 non-null  float64
 6   cod_material_piso_fam       296062 non-null  float64
 7   cod_material_domic_fam      296062 non-null  float64
 8   cod_agua_canalizada_fam     296062 non-null  float64
 9   cod_abaste_agua_domic_fam   296062 non-null  float64
 10  cod_banheiro_domic_fam      296062 non-null  float64
 11  cod_destino_lixo_domic_fam  296062 non-null  float64
 12  cod_iluminacao_domic_fam    296062 non-null  float64
 13  cod_calcamento

In [ ]:
treated_data_RN.head()

In [7]:
list(treated_data_RN.columns)

['Unnamed: 0',
 'classf',
 'vlr_renda_media_fam',
 'cod_local_domic_fam',
 'qtd_comodos_domic_fam',
 'qtd_comodos_dormitorio_fam',
 'cod_material_piso_fam',
 'cod_material_domic_fam',
 'cod_agua_canalizada_fam',
 'cod_abaste_agua_domic_fam',
 'cod_banheiro_domic_fam',
 'cod_destino_lixo_domic_fam',
 'cod_iluminacao_domic_fam',
 'cod_calcamento_domic_fam',
 'ind_familia_quilombola_fam',
 'marc_pbf',
 'qtde_pessoas']

In [9]:
final_features = ['vlr_renda_media_fam',
'qtd_comodos_domic_fam',
'cod_material_piso_fam',
'cod_agua_canalizada_fam',
'cod_abaste_agua_domic_fam',
'cod_calcamento_domic_fam',
'qtde_pessoas',
'marc_pbf' ]

treated_data_RN = treated_data_RN[final_features]

treated_data_RN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296062 entries, 0 to 296061
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   vlr_renda_media_fam        296062 non-null  float64
 1   qtd_comodos_domic_fam      296062 non-null  float64
 2   cod_material_piso_fam      296062 non-null  float64
 3   cod_agua_canalizada_fam    296062 non-null  float64
 4   cod_abaste_agua_domic_fam  296062 non-null  float64
 5   cod_calcamento_domic_fam   296062 non-null  float64
 6   qtde_pessoas               296062 non-null  int64  
 7   marc_pbf                   296062 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 18.1 MB


In [16]:
treated_data_RN.head()

,vlr_renda_media_fam,qtd_comodos_domic_fam,cod_material_piso_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_calcamento_domic_fam,qtde_pessoas,marc_pbf
0,312.0,4.0,2.0,1.0,1.0,1.0,3,0
1,75.0,4.0,5.0,2.0,4.0,1.0,2,1
2,60.0,5.0,2.0,1.0,1.0,3.0,5,1
3,394.0,5.0,5.0,1.0,1.0,1.0,2,0
4,66.0,5.0,2.0,1.0,1.0,1.0,3,1


### Treinando o modelo XGBoost

In [10]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_RN['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution, 2))

marc_pbf
1    56.74
0    43.26
Name: proportion, dtype: float64


In [11]:
# separar X (features) e y (target)
X = treated_data_RN.drop('marc_pbf', axis=1)
y = treated_data_RN['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(y_resampled.value_counts())

marc_pbf
0    89679
1    89679
Name: count, dtype: int64


In [12]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[-0.12485031 -0.602914   -0.72537277 -0.39178363 -0.45627415 -0.85437521
  -1.37479098]
 [ 0.13504909 -0.602914   -0.72537277 -0.39178363 -0.45627415  1.22999908
   0.90668609]
 [ 4.64343735  0.1538476   1.28227208 -0.39178363  1.7490837   1.22999908
  -1.37479098]
 [ 1.40335816  3.93765562  1.28227208 -0.39178363 -0.45627415  0.18781194
   0.14619374]
 [-0.81791538  0.91060921  1.28227208 -0.39178363 -0.45627415 -0.85437521
   0.14619374]]


In [13]:
final_model = xgb.XGBClassifier()
final_model.fit(X_train_scaled, y_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [14]:
# Faça previsões nos dados de teste
y_pred = final_model.predict(X_test_scaled)

# Imprima o relatório de classificação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.82      0.87     38411
           1       0.88      0.95      0.91     50408

    accuracy                           0.90     88819
   macro avg       0.90      0.89      0.89     88819
weighted avg       0.90      0.90      0.90     88819



In [ ]:
joblib.dump(final_model, '/home/erlon/AdministracaoPublica/MlApi/model/XGBoostRNModelReduced.joblib')

['/home/erlon/AdministracaoPublica/MlApi/XGBoostRNModelReduced.joblib']

### Selecionando features com SelectKBests

In [ ]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

In [ ]:
# Treine o modelo final com o melhor valor de k
final_model = xgb.XGBClassifier()
final_model.fit(X_train_selected, y_resampled)

In [ ]:
# Faça previsões nos dados de teste
y_pred_selected = final_model.predict(X_test_selected)

# Imprima o relatório de classificação
print(classification_report(y_test, y_pred_selected))